In [ ]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM(), simulation=False)
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(.1)

    time.sleep(3)     
        
    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:40641
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:40109
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:43103
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:53885
INFO:root:127.0.0.1:40641 Conexión aceptada con 127.0.0.1:40109
INFO:root:127.0.0.1:40109 Connected to 127.0.0.1:40641
INFO:root:127.0.0.1:43103 Connected to 127.0.0.1:40109
INFO:root:127.0.0.1:40109 Conexión aceptada con 127.0.0.1:43103
INFO:root:127.0.0.1:40641 Connected to 127.0.0.1:43103
INFO:root:127.0.0.1:43103 Conexión aceptada con 127.0.0.1:40641
INFO:root:127.0.0.1:53885 Connected to 127.0.0.1:43103
INFO:root:127.0.0.1:43103 Conexión aceptada con 127.0.0.1:53885
INFO:root:127.0.0.1:40109 Connected to 127.0.0.1:53885
INFO:root:127.0.0.1:53885 Conexión aceptada con 127.0.0.1:40109
INFO:root:127.0.0.1:40641 Connected to 127.0.0.1:53885
INFO:root:127.0.0.1:53885 Conexión aceptad

INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:40641) Sending train set vote.
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:43103) Sending train set vote.
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:40641) Self Vote: [('127.0.0.1:53885', 521), ('127.0.0.1:43103', 37), ('127.0.0.1:40641', 155)]
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:43103) Self Vote: [('127.0.0.1:40109', 698), ('127.0.0.1:53885', 260), ('127.0.0.1:40641', 131)]
INFO:root:(127.0.0.1:53885) Gossip finished.
INFO:root:(127.0.0.1:40641) Waiting other node votes.
INFO:root:(127.0.0.1:40109) Gossip finished.
INFO:root:(127.0.0.1:43103) Waiting other node votes.
INFO:root:(127.0.0.1:53885) Round 1 of 2 finished.
INFO:root:(127.0.0.1:40109) Round 1 of 2 finished.
INFO:root:(127.0.0.1:53885) Sending train set vote.
INFO:root:(127.0.0.1:40109) Sending train set vote.
INFO:root:(127.0.0.1:53885) Self Vote: [('127.0.0.1:43103', 739), ('127.0.0.1:40109', 97), ('127.0.0.1:40641', 204)]


In [7]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(7)
    nodes[1].stop()

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:58421
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:59085
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:36879
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:58421
INFO:root:127.0.0.1:58421 Conexión aceptada con 127.0.0.1:59085
INFO:root:127.0.0.1:36879 Connected to 127.0.0.1:59085
INFO:root:127.0.0.1:59085 Conexión aceptada con 127.0.0.1:36879
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:54231
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:54231 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Conexión aceptad

INFO:root:(127.0.0.1:54231) Round 1 of 2 finished.
INFO:root:(127.0.0.1:58421) Self Vote: [('127.0.0.1:58421', 76), ('127.0.0.1:54231', 476), ('127.0.0.1:36879', 156)]
INFO:root:(127.0.0.1:36879) Round 1 of 2 finished.
INFO:root:(127.0.0.1:54231) Sending train set vote.
INFO:root:(127.0.0.1:58421) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Sending train set vote.
INFO:root:(127.0.0.1:54231) Self Vote: [('127.0.0.1:36879', 180), ('127.0.0.1:59085', 240), ('127.0.0.1:58421', 193)]
INFO:root:(127.0.0.1:36879) Self Vote: [('127.0.0.1:58421', 23), ('127.0.0.1:59085', 319), ('127.0.0.1:54231', 71)]
INFO:root:(127.0.0.1:54231) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Waiting other node votes.
INFO:root:(127.0.0.1:58421) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:36879) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:54231) Timeout for v

In [1]:
import pytest
from p2pfl.base_node import BaseNode
from p2pfl.communication_protocol import CommunicationProtocol
import time
import pytest
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
from p2pfl.settings import Settings


def four_nodes():
    n1 = Node(MLP(),MnistFederatedDM())
    n2 = Node(MLP(),MnistFederatedDM())
    n3 = Node(MLP(),MnistFederatedDM())
    n4 = Node(MLP(),MnistFederatedDM())
    n1.start()
    n2.start()
    n3.start()
    n4.start()

    return n1,n2,n3,n4

def test_gossip_heartbeat():

    n1.connect_to(n2.host,n2.port, full=False)
    n2.connect_to(n3.host,n3.port, full=False)
    n3.connect_to(n4.host,n4.port, full=False)

    time.sleep(2) # Wait for asincronity    

    assert len(n1.heartbeater.get_nodes()) == len(n2.heartbeater.get_nodes()) == len(n3.heartbeater.get_nodes()) == len(n4.heartbeater.get_nodes()) == 3

    
    n1.set_start_learning(rounds=2,epochs=0)    

    while all([n1.round is not None for n in [n1,n2,n3,n4]]):
        time.sleep(0.1)

    for n in [n1,n2,n3,n4]:
        n.stop()
        time.sleep(0.1)

n1, n2, n3, n4 = four_nodes()
test_gossip_heartbeat()

Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:37107
INFO:root:Nodo listening at 127.0.0.1:37339
INFO:root:Nodo listening at 127.0.0.1:40133
INFO:root:Nodo listening at 127.0.0.1:55395
INFO:root:127.0.0.1:37107 Connected to 127.0.0.1:37339
INFO:root:127.0.0.1:37339 Conexión aceptada con 127.0.0.1:37107
INFO:root:127.0.0.1:37339 Connected to 127.0.0.1:40133
INFO:root:127.0.0.1:40133 Conexión aceptada con 127.0.0.1:37339
INFO:root:127.0.0.1:40133 Connected to 127.0.0.1:55395
INFO:root:127.0.0.1:55395 Conexión aceptada con 127.0.0.1:40133
INFO:root:(127.0.0.1:37107) Broadcasting start learning...
INFO:root:(127.0.0.1:37339) Waiting initialization.
INFO:root:(127.0.0.1:37107) Waiting initialization.
INFO:root:(127.0.0.1:37107) Gossiping model initialization.
INFO:root:(127.0.0.1:37107) Gossiping model to 1 train set nodes.
INFO:root:(127.0.0.1:40133) Waiting initializa

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:40133) Model added (3/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (3/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:37339) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Model added (4/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:55395) Agregating models.
INFO:root:(127.0.0.1:40133) Model added (4/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:40133) Agregating models.
INFO:root:(127.0.0.1:37107) Model added (3/4) from ['127.0.0.1:55395', '127.0.0.1:40133']
DEBUG:root:(127.0.0.1:37107) Can't add a model tha

DEBUG:root:Closed connection: 127.0.0.1:40133
DEBUG:root:(127.0.0.1:55395) (NodeConnection Loop) Exception: [Errno 104] Connection reset by peer
DEBUG:root:Closed connection: 127.0.0.1:55395
INFO:root:(127.0.0.1:40133) Evaluating...
INFO:root:(127.0.0.1:40133) Training...
INFO:root:(127.0.0.1:55395) Evaluating...
INFO:root:(127.0.0.1:40133) Model added (1/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Training...
INFO:root:(127.0.0.1:40133) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:37339) Votes: {'127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'} of {'127.0.0.1:37339', '127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'}
INFO:root:(127.0.0.1:37107) Votes: {'127.0.0.1:37339', '

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37339) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37339) Evaluating...
INFO:root:(127.0.0.1:37339) Taining finished!!.
ERROR:root:(127.0.0.1:37339) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37107) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37107) Evaluating...
INFO:root:(127.0.0.1:37107) Taining finished!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 37107 y desconectándose de 3 nodos
DEBUG:root:Closed connection: 127.0.0.1:37107
DEBUG:root:Closed connection: 127.0.0.1:37107

In [3]:
print(len(n1.heartbeater.get_nodes()) , len(n2.heartbeater.get_nodes()), len(n3.heartbeater.get_nodes()) , len(n4.heartbeater.get_nodes()))


3 3 3 3
DEBUG:root:{} | {'127.0.0.1:50847': {'127.0.0.1:50847': 418, '127.0.0.1:43679': 316, '127.0.0.1:43977': 130}}
DEBUG:root:{'127.0.0.1:50847': {'127.0.0.1:50847': 418, '127.0.0.1:43679': 316, '127.0.0.1:43977': 130}} | {'127.0.0.1:43977': {'127.0.0.1:43977': 568, '127.0.0.1:43679': 306, '127.0.0.1:45389': 159}}
DEBUG:root:{} | {'127.0.0.1:45389': {'127.0.0.1:45389': 397, '127.0.0.1:43679': 217, '127.0.0.1:50847': 200}}
INFO:root: Votes agregated from: ['127.0.0.1:50847', '127.0.0.1:43977']
wwwwwwwwwwwwwww False
INFO:root: Votes agregated from: ['127.0.0.1:45389']


Exception in thread learning_thread-127.0.0.1:43977:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
Exception in thread learning_thread-127.0.0.1:45389:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 292, in __start_learning
    self.__train_step()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 388, in __train_step
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self.__validate_train_set()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 569, in __validate_train_set
    self._target(*self._args, **self._kwargs)
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 292, in __sta

wwwwwwwwwwwwwww False
DEBUG:root:{} | {'127.0.0.1:43977': {'127.0.0.1:43977': 568, '127.0.0.1:43679': 306, '127.0.0.1:45389': 159}}
INFO:root: Votes agregated from: ['127.0.0.1:43977']
wwwwwwwwwwwwwww False
DEBUG:root:{} | {'127.0.0.1:45389': {'127.0.0.1:45389': 397, '127.0.0.1:43679': 217, '127.0.0.1:50847': 200}}
DEBUG:root:{'127.0.0.1:45389': {'127.0.0.1:45389': 397, '127.0.0.1:43679': 217, '127.0.0.1:50847': 200}} | {'127.0.0.1:45389': {'127.0.0.1:45389': 669, '127.0.0.1:50847': 288, '127.0.0.1:43977': 22}}
INFO:root: Votes agregated from: ['127.0.0.1:45389']


Exception in thread learning_thread-127.0.0.1:43679:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
Exception in thread learning_thread-127.0.0.1:50847:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 292, in __start_learning
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self.__train_step()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 388, in __train_step
    self.__validate_train_set()
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 569, in __validate_train_set
    self._target(*self._args, **self._kwargs)
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node.py", line 292, in __sta

wwwwwwwwwwwwwww False


In [4]:
a = {"a":1,"b":2}
b = {"a":2,"b":1}

{**a,**b}

{'a': 2, 'b': 1}

In [2]:
set([1,2,3]) in set([1,2,3])

False

In [5]:
a ={'127.0.0.1:43617', '127.0.0.1:49487', '127.0.0.1:46509', '127.0.0.1:48487'} 
b = {'127.0.0.1:43617', '127.0.0.1:49487', '127.0.0.1:46509', '127.0.0.1:48487'}

a == b

True

In [4]:
import pytest
from p2pfl.base_node import BaseNode
from p2pfl.communication_protocol import CommunicationProtocol
import time
import pytest
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
from p2pfl.settings import Settings


def four_nodes():
    n1 = Node(MLP(),MnistFederatedDM())
    n2 = Node(MLP(),MnistFederatedDM())
    n3 = Node(MLP(),MnistFederatedDM())
    n4 = Node(MLP(),MnistFederatedDM())
    n1.start()
    n2.start()
    n3.start()
    n4.start()


    # RED ANILLO
    n1.connect_to(n2.host,n2.port, full=False)
    n2.connect_to(n3.host,n3.port, full=False)
    n3.connect_to(n4.host,n4.port, full=False)
    n4.connect_to(n1.host,n1.port, full=False)

    acum = 0
    while len(n1.heartbeater.get_nodes()) != len(n2.heartbeater.get_nodes()) != len(n3.heartbeater.get_nodes()) != len(n4.heartbeater.get_nodes()) != 4:
        begin = time.time()
        time.sleep(0.1)
        acum += time.time() - begin
        if acum > 6:
            assert False
    
    time.sleep(2)
    
    print(len(n1.get_neighbors()))
    print(len(n2.get_neighbors()))
    print(len(n3.get_neighbors()))
    print(len(n4.get_neighbors()))
    assert len(n1.get_neighbors()) == len(n2.get_neighbors()) == len(n3.get_neighbors()) == len(n4.get_neighbors()) == 2

    n1.set_start_learning(rounds=2,epochs=0)  

    while all([n.round is not None for n in four_nodes]):
        time.sleep(0.1)

    assert len(n1.heartbeater.get_nodes()) == len(n2.heartbeater.get_nodes()) == len(n3.heartbeater.get_nodes()) == len(n4.heartbeater.get_nodes()) == 4
    assert len(n1.get_neighbors()) == len(n2.get_neighbors()) == len(n3.get_neighbors()) == len(n4.get_neighbors()) == 2

    for n in four_nodes:
        n.stop()
four_nodes()

Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:36553
INFO:root:Nodo listening at 127.0.0.1:38503
INFO:root:Nodo listening at 127.0.0.1:57921
INFO:root:Nodo listening at 127.0.0.1:59679
INFO:root:127.0.0.1:36553 Connected to 127.0.0.1:38503
INFO:root:127.0.0.1:38503 Conexión aceptada con 127.0.0.1:36553
INFO:root:127.0.0.1:38503 Connected to 127.0.0.1:57921
INFO:root:127.0.0.1:57921 Conexión aceptada con 127.0.0.1:38503
INFO:root:127.0.0.1:57921 Connected to 127.0.0.1:59679
INFO:root:127.0.0.1:59679 Conexión aceptada con 127.0.0.1:57921
INFO:root:127.0.0.1:59679 Connected to 127.0.0.1:36553
INFO:root:127.0.0.1:36553 Conexión aceptada con 127.0.0.1:59679
2
2
2
2
INFO:root:(127.0.0.1:36553) Broadcasting start learning...
INFO:root:(127.0.0.1:36553) Waiting initialization.
INFO:root:(127.0.0.1:59679) Waiting initialization.


TypeError: 'function' object is not iterable

INFO:root:(127.0.0.1:57921) Waiting initialization.
INFO:root:(127.0.0.1:36553) Gossiping model initialization.
INFO:root:(127.0.0.1:38503) Waiting initialization.
INFO:root:(127.0.0.1:36553) Gossiping model to 2 nodes set nodes.
INFO:root:(127.0.0.1:38503) Initialicing Model Weights
INFO:root:(127.0.0.1:38503) Gossiping model initialization.
INFO:root:(127.0.0.1:38503) Gossiping model to 1 nodes set nodes.
INFO:root:(127.0.0.1:59679) Initialicing Model Weights
INFO:root:(127.0.0.1:59679) Gossiping model initialization.
INFO:root:(127.0.0.1:59679) Gossiping model to 1 nodes set nodes.
INFO:root:(127.0.0.1:57921) Initialicing Model Weights
INFO:root:(127.0.0.1:57921) Gossiping model initialization.
INFO:root:(127.0.0.1:57921) Gossip finished.
INFO:root:(127.0.0.1:36553) Gossip finished.
INFO:root:(127.0.0.1:38503) Gossip finished.
INFO:root:(127.0.0.1:59679) Gossip finished.
INFO:root:(127.0.0.1:57921) Sending train set vote.
INFO:root:(127.0.0.1:57921) Self Vote: [('127.0.0.1:57921', 4

DEBUG:root:Closed connection: 127.0.0.1:57921
INFO:root:(127.0.0.1:57921) Sending train set vote.
DEBUG:root:Closed connection: 127.0.0.1:36553
INFO:root:(127.0.0.1:57921) Self Vote: [('127.0.0.1:38503', 5), ('127.0.0.1:57921', 388), ('127.0.0.1:36553', 168)]
INFO:root:(127.0.0.1:57921) Waiting other node votes.
INFO:root:(127.0.0.1:36553) Gossip finished.
INFO:root:(127.0.0.1:36553) Waiting aregation.
INFO:root:(127.0.0.1:36553) Gossiping agregated model.
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:36553) Gossip finished.
INFO:root:(127.0.0.1:36553) Round 1 of 2 finished.
INFO:root:(127.0.0.1:36553) Sending train set vote.
INFO:root:(127.0.0.1:36553) Self Vote: [('127.0.0.1:36553', 715), ('127.0.0.1:38503', 362), ('127.0.0.1:57921', 161)]
INFO:root:(127.0.0.1:36553) Waiting other node votes.
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:NC STAT: 0 NODE ROUND: 0
INFO:root:(127.0.0.1:38503) Gossip finished.
INFO:root:(127.0.0.1:38503) Round 

INFO:root:(127.0.0.1:59679) Taining finished!!.


In [1]:
3 != 3 != 3 != 3 != 4

False

In [2]:
all([1,2,3,4])

True